In [320]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer

In [329]:
df = pd.read_csv('data.csv')
df = df.drop(columns=['name','smiles'])
df = df.dropna()
df

<ipython-input-329-38799959fd12>:1: DtypeWarning: Columns (1147) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data.csv')


,output,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,nB,nC,nN,nO,nS,nP,nF,nCl,nBr,nI,nX,ATS0m,ATS1m,ATS2m,ATS3m,ATS4m,ATS5m,ATS6m,ATS7m,ATS8m,ATS0v,ATS1v,ATS2v,ATS3v,ATS4v,ATS5v,ATS6v,ATS7v,ATS8v,ATS0e,ATS1e,ATS2e,ATS3e,ATS4e,ATS5e,ATS6e,ATS7e,ATS8e,ATS0p,...,L3e,P1e,P2e,E1e,E2e,E3e,Te,Ae,Ve,Ke,De,L1p,L2p,L3p,P1p,P2p,E1p,E2p,E3p,Tp,Ap,Vp,Kp,Dp,L1i,L2i,L3i,P1i,P2i,E1i,E2i,E3i,Ti,Ai,Vi,Ki,Di,L1s,L2s,L3s,P1s,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds
0,0,0,-1.8653,3.479344,47.8891,44.044274,9,10,39,21,18,0,14,6,1,0,0,0,0,0,0,0,3471.131141,4025.909636,5672.706036,5616.401151,5777.223856,5114.319496,4518.242120,3209.421483,2157.304791,8165.019841,10762.764780,15434.88105,15986.394170,15952.631300,15178.689510,13712.815400,10778.966540,8044.452639,301.061108,332.182808,571.237688,713.546956,694.778928,665.885800,612.759504,599.928516,468.966396,54.950836,...,0.432127,0.862286,0.116713,0.553898,0.373634,0.261353,20.576626,51.315729,90.305612,0.793429,1.188885,16.287060,2.018843,0.318874,0.874483,0.108396,0.466376,0.260971,0.140026,18.624777,38.718291,67.827961,0.811725,0.867373,17.973216,2.522212,0.473711,0.857127,0.120282,0.568662,0.414292,0.313981,20.969139,55.041172,97.484709,0.785691,1.296935,17.850239,2.419192,0.437409,0.862046,0.116831,0.560654,0.379450,0.268039,20.706840,52.049186,91.644726,0.793068,1.208143
1,0,0,-7.2263,52.219412,137.7680,88.208099,0,0,87,44,43,0,25,1,18,0,0,0,0,0,0,0,8453.913844,8504.934067,13493.959710,18053.568930,16811.910920,16174.629820,17117.705210,17066.907070,14514.604770,16062.871710,21402.676400,33637.36769,41992.793440,42885.709300,42677.463230,44100.590550,43578.629860,37424.366780,727.939376,752.027492,1379.739568,2122.919528,2394.060096,2436.334544,2440.267884,2545.023456,2508.419684,101.628527,...,1.907018,0.867095,0.085798,0.577559,0.440367,0.459437,40.482312,195.483653,468.469361,0.800642,1.477363,33.868877,3.171001,1.578485,0.877015,0.082111,0.537788,0.364807,0.311452,38.618363,165.865123,374.009984,0.815522,1.214046,35.037954,3.499403,1.929415,0.865845,0.086476,0.575465,0.446090,0.469301,40.466771,196.966453,474.002453,0.798768,1.490857,35.394342,3.484472,1.932554,0.867267,0.085380,0.587220,0.443947,0.472578,40.811368,198.465988,477.620358,0.800900,1.503745
2,0,0,-2.1004,4.411680,67.0727,55.758204,6,6,52,24,28,0,18,2,4,0,0,0,0,0,0,0,4041.468072,4144.776372,5658.235903,6001.338911,5918.481335,5768.719826,5211.769774,5038.451937,4017.184500,9845.840530,12274.803950,16832.63403,18658.863340,18332.715630,18243.315900,17502.712540,15694.713070,13950.553600,397.656416,402.329720,703.547668,925.953556,938.030540,883.768992,906.352984,896.066900,737.278056,67.642177,...,0.939740,0.910911,0.056955,0.490177,0.314093,0.381758,29.244869,70.971504,141.914546,0.866367,1.186028,25.347516,1.444408,0.730191,0.920987,0.052482,0.443908,0.237756,0.232241,27.522115,56.175374,110.431351,0.881481,0.913905,27.624605,1.730722,0.954571,0.911405,0.057101,0.527108,0.339235,0.394121,30.309898,75.832250,151.780675,0.867108,1.260464,26.749160,1.678309,0.942758,0.910758,0.057143,0.494202,0.319559,0.386393,29.370226,71.693573,143.387359,0.866137,1.200154
3,0,0,-0.5179,0.268220,52.6108,46.640239,6,6,43,20,23,0,15,3,2,0,0,0,0,0,0,0,3287.855436,3431.386808,4531.128985,4609.877881,4282.627466,3632.420586,3125.200404,3007.402544,2936.316999,8230.403722,10094.618880,13565.35759,15178.141670,14906.660720,13224.139620,10883.181600,9672.171432,9387.042086,324.940752,330.436480,571.584164,718.387492,740.634800,792.880168,653.872544,499.853344,430.708812,56.976005,...,1.023171,0.908773,0.048679,0.531044,0.381209,0.459508,24.047227,49.138997,99.360424,0.863159,1.371761,20.754335,0.977458,0.819781,0.920305,0.043343,0.479157,0.264849,0.297465,22.551573,38.101789,77.283830,0.880458,1.041471,22.243774,1.232167,1.047641,0.907036,0.050244,0.550381,0.426016,0.482330,24.523582,52.002397,105.239763,0.860554,1.458727,21.913407,1.181143,1.034116,0.908190,0.048952,0.533958,0.387559,0.470449,24.128665,49.765298,100.659835,0.862285,1.

In [330]:
#複製一份只有feature的data(結構：pandas)
data = df
data = data.drop(columns=['output'])
selected_features = data.columns

In [331]:
#去除極端值
threshold = 0.95
data = data.apply(pd.to_numeric, errors='coerce')
extreme_values = data.apply(lambda col: col > col.quantile(threshold),axis=0)
replacement_value = data.quantile(0.95)
for col in data.columns:
    data.loc[extreme_values[col], col] = replacement_value[col]


In [332]:
#前處理（標準化 Standardization）
scaler = StandardScaler()
data = scaler.fit_transform(data)

In [286]:
#前處理（最大最小縮放 Min-Max Scaling）
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

In [264]:
#前處理（Robust Scaling）
scaler = RobustScaler()
data = scaler.fit_transform(data)

In [281]:
#前處理（Normalization）
scaler = Normalizer()
data = scaler.fit_transform(data)

In [240]:
data = pd.DataFrame(data,columns=selected_features)

#特徵選擇（相關係數法）
correlations = data.corrwith(df['output'])

r_threshold = 0.2
selected_features = correlations[correlations.abs() > r_threshold].index


<ipython-input-240-c6c4e29ce825>:4: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlations = data.corrwith(df['output'])


In [242]:
#特徵選擇（決策樹法）
tree = DecisionTreeClassifier()
tree.fit(data, df['output'])
importances = pd.DataFrame({'features':selected_features, 'importance':tree.feature_importances_}).sort_values('importance', ascending=False)

#從決策樹的信息熵來選擇
sfm = SelectFromModel(tree, threshold=.05)
data = sfm.fit_transform(data, df['output'])

In [ ]:
data

array([[0.257485854, 0.0, 2.892934402],
       [0.0, 0.0, 12.55626332],
       [0.382701142, 0.0, 2.109226624],
       ...,
       [0.187124223, 2.148889201, 3.148231892],
       [1.593779972, 2.211810794, 0.612291693],
       [1.609318809, 2.20918286, 1.163684742]], dtype=object)

In [ ]:
importances

,features,importance
1082,maxsssN,0.209173
1622,RDF100m,0.097898
950,minaasC,0.061941
1370,nRotB,0.044395
210,AATSC0v,0.037601
...,...,...
639,Spe,0.000000
638,Sse,0.000000
637,Sv,0.000000
636,AVP-7,0.000000


In [ ]:
#主成份分析
com_number = 10
pca = PCA(n_components=com_number)

pca_result = pca.fit_transform(data)
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)
print("Explained variance ratio:", explained_variance)
print("Cumulative variance ratio:", cumulative_variance)

In [333]:
#使用支持向量機
#分割訓練集與測試集：使用data（不降維）
x_train, x_test, y_train, y_test = train_test_split(data, df['output'], test_size=0.2, random_state=42)
print('資料量:', x_train.shape, x_test.shape, y_train.size, y_test.size)

#創建Support Vector Classifier
model = svm.SVC(kernel='rbf',C=3)

#模型訓練
model.fit(x_train, y_train)

#預測資料
y_pred = model.predict(x_test)

#評估模型
accuracy = accuracy_score(y_test, y_pred)
accuracy

資料量: (600, 1875) (151, 1875) 600 151


0.8874172185430463

In [295]:
#分割訓練集與測試集：使用pca_result
x_train, x_test, y_train, y_test = train_test_split(pca_result, df['output'], test_size=0.2, random_state=42)
print('資料量:', x_train.shape, x_test.shape, y_train.size, y_test.size)

#創建Support Vector Classifier
model = svm.SVC(kernel='rbf')

#模型訓練
model.fit(x_train, y_train)

#預測資料
y_pred = model.predict(x_test)

#評估模型
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

資料量: (600, 500) (151, 500) 600 151
0.8675496688741722
